## **PHẦN 1 – IMPORT + CONFIG**

In [ ]:
# Chạy trong cmd
conda init cmd.exe


conda activate
(base) C:\Users\cchau>

conda activate seq2seq
(seq2seq) C:\Users\cchau>

conda create -n seq2seq python=3.9 -y
conda activate seq2seq

pip install numpy matplotlib tqdm spacy nltk
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm

pip install ipykernel
python -m ipykernel install --user --name seq2seq --display-name "Python (seq2seq)"

Python (seq2seq)

# Chạy trong terminal
conda create -n seq2seq python=3.9 -y
conda activate seq2seq
conda activate seq2seq

pip install torch==1.13.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

conda install -c conda-forge spacy
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm
conda install -c conda-forge spacy
pip install ipykernel
python -m ipykernel install --user --name seq2seq --display-name "Python (seq2seq)"
pip install matplotlib
pip install numpy tqdm nltk


In [ ]:
import os
import requests

# Tạo thư mục data nếu chưa có
os.makedirs("data", exist_ok=True)

# File EN-DE từ Multi30K chuẩn (task1/tok)
files_map = {
    "train.en": "train.lc.norm.tok.en",
    "train.de": "train.lc.norm.tok.de",
    "val.en":   "val.lc.norm.tok.en",
    "val.de":   "val.lc.norm.tok.de",
    "test.en":  "test_2016_flickr.lc.norm.tok.en",
    "test.de":  "test_2016_flickr.lc.norm.tok.de"
}

base_url = "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/tok/"

def download_file(fname, real_name):
    url = base_url + real_name
    save_path = os.path.join("data", fname)

    print(f"Downloading {fname} ← {real_name}")
    r = requests.get(url)

    if r.status_code == 200:
        with open(save_path, "wb") as f:
            f.write(r.content)
        print(f"✔ Saved: data/{fname}")
    else:
        print(f"❌ Failed ({r.status_code}): {url}")

# Tải file
for fname, real_name in files_map.items():
    download_file(fname, real_name)

print("\n🎉 DONE — Multi30K EN–DE đã tải hoàn tất!")


✔ Saved: data/train.en
✔ Saved: data/train.de
✔ Saved: data/val.en
✔ Saved: data/val.de
✔ Saved: data/test.en
✔ Saved: data/test.de

🎉 DONE — Multi30K EN–DE đã tải hoàn tất!


In [2]:
# ============================================================
# PHẦN 1 – IMPORT & CONFIG (EN–DE)
# ============================================================

import os
os.environ["TORCHINDUCTOR_DISABLE"] = "1"   # tắt Inductor
os.environ["TORCH_COMPILE_DISABLE"] = "1"   # tắt torch.compile

import torch
try:
    torch._dynamo.config.suppress_errors = True
    torch._dynamo.disable()
except:
    pass
import random
import math
import time
import spacy
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# Seed cố định
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed(SEED)

# Các token đặc biệt
SOS_TOKEN = "<sos>"
EOS_TOKEN = "<eos>"
PAD_TOKEN = "<pad>"
UNK_TOKEN = "<unk>"

SPECIAL_TOKENS = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN]

Using device: cpu


C:\Users\cchau\AppData\Local\Temp\ipykernel_1308\664502001.py:30: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: The specified module could not be found. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:77.)
  DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## **PHẦN 2 – TẢI DỮ LIỆU MULTI30K (EN–DE)**

In [3]:
# ============================================================
# PHẦN 2 – LOAD RAW MULTI30K EN–DE
# ============================================================

DATA_DIR = "data"

def load_lines(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f.readlines()]

# Load English–German
train_en = load_lines(os.path.join(DATA_DIR, "train.en"))
train_de = load_lines(os.path.join(DATA_DIR, "train.de"))

val_en   = load_lines(os.path.join(DATA_DIR, "val.en"))
val_de   = load_lines(os.path.join(DATA_DIR, "val.de"))

test_en  = load_lines(os.path.join(DATA_DIR, "test.en"))
test_de  = load_lines(os.path.join(DATA_DIR, "test.de"))

print("Train size:", len(train_en))
print("Val size:", len(val_en))
print("Test size:", len(test_en))


Train size: 29000
Val size: 1014
Test size: 1000


## **PHẦN 3 – TOKENIZER**

In [4]:
# ============================================================
# PHẦN 3 – TOKENIZER (EN–DE)
# ============================================================

!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

spacy_en = spacy.load("en_core_web_sm")
spacy_de = spacy.load("de_core_news_sm")

def tokenize_en(text):
    return [tok.text.lower() for tok in spacy_en.tokenizer(text)]

def tokenize_de(text):
    return [tok.text.lower() for tok in spacy_de.tokenizer(text)]


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 5.6 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 6.3 MB/s eta 0:00:02
     --------- ------------------------------ 2.9/12.8 MB 5.6 MB/s eta 0:00:02
     ------------ --------------------------- 3.9/12.8 MB 5.5 MB/s eta 0:00:02
     ---------------- ----------------------- 5.2/12.8 MB 5.5 MB/s eta 0:00:02
     ------------------- -------------------- 6.3/12.8 MB 5.5 MB/s eta 0:00:02
     ----------------------- ---------------- 7.6/12.8 MB 5.6 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 5.7 MB/s eta 0:00:01
     ------------------------------- -------- 10.0/12.8 MB 5.6 MB/s eta 0:00:01
     ----------------------------------- ---- 11.3/12.8 MB 5.7 MB/s eta 0:00:01
     ---------------------------------------  12.6/12.8 MB 5.8 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 

## **PHẦN 4 – BUILD VOCAB (EN–DE)**

In [5]:
# ============================================================
# PHẦN 4 – BUILD VOCAB (English–German)
# ============================================================

from collections import Counter

def build_vocab(texts, tokenizer, max_size=10000):
    counter = Counter()
    for sent in texts:
        counter.update(tokenizer(sent))

    vocab = SPECIAL_TOKENS + [w for w, _ in counter.most_common(max_size)]
    
    word2idx = {w: i for i, w in enumerate(vocab)}
    idx2word = {i: w for i, w in enumerate(vocab)}
    
    return word2idx, idx2word

# Vocab English
en_word2idx, en_idx2word = build_vocab(train_en, tokenize_en)

# Vocab German
de_word2idx, de_idx2word = build_vocab(train_de, tokenize_de)

PAD_IDX = en_word2idx[PAD_TOKEN]
SOS_IDX = en_word2idx[SOS_TOKEN]
EOS_IDX = en_word2idx[EOS_TOKEN]
UNK_IDX = en_word2idx[UNK_TOKEN]

print("EN vocab size:", len(en_word2idx))
print("DE vocab size:", len(de_word2idx))

EN vocab size: 9789
DE vocab size: 10004


## **PHẦN 5 – Dataset + DataLoader (EN–DE)**

In [6]:
# ============================================================
# PHẦN 5 – DATASET + DATALOADER (EN–DE)
# ============================================================

class TranslationDataset(Dataset):
    def __init__(self, src_texts, trg_texts, src_tokenizer, trg_tokenizer, 
                 src_w2i, trg_w2i):
        self.src = src_texts
        self.trg = trg_texts
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = trg_tokenizer
        self.src_w2i = src_w2i
        self.trg_w2i = trg_w2i

    def __len__(self):
        return len(self.src)

    def numericalize(self, tokens, w2i):
        return [w2i.get(tok, w2i[UNK_TOKEN]) for tok in tokens]

    def __getitem__(self, idx):
        src_sent = self.src[idx]
        trg_sent = self.trg[idx]

        src_tok = self.src_tokenizer(src_sent)
        trg_tok = self.trg_tokenizer(trg_sent)

        # add SOS / EOS cho target
        trg_tok = [SOS_TOKEN] + trg_tok + [EOS_TOKEN]

        src_ids = self.numericalize(src_tok, self.src_w2i)
        trg_ids = self.numericalize(trg_tok, self.trg_w2i)

        return torch.tensor(src_ids), torch.tensor(trg_ids)


def collate_fn(batch):
    src_batch = [item[0] for item in batch]
    trg_batch = [item[1] for item in batch]

    src_lens = torch.tensor([len(x) for x in src_batch])
    trg_lens = torch.tensor([len(x) for x in trg_batch])

    # sort theo độ dài giảm dần (bắt buộc cho pack_padded)
    src_lens, idx = src_lens.sort(descending=True)
    src_batch = [src_batch[i] for i in idx]
    trg_batch = [trg_batch[i] for i in idx]
    trg_lens = trg_lens[idx]

    src_pad = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    trg_pad = pad_sequence(trg_batch, padding_value=PAD_IDX, batch_first=True)

    return src_pad, trg_pad, src_lens, trg_lens


# Tạo dataset
train_ds = TranslationDataset(train_en, train_de, tokenize_en, tokenize_de,
                              en_word2idx, de_word2idx)
val_ds = TranslationDataset(val_en, val_de, tokenize_en, tokenize_de,
                            en_word2idx, de_word2idx)
test_ds = TranslationDataset(test_en, test_de, tokenize_en, tokenize_de,
                             en_word2idx, de_word2idx)

# DataLoader
BATCH_SIZE = 64

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate_fn)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                        collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                         collate_fn=collate_fn)

len(train_loader), len(val_loader), len(test_loader)


(454, 16, 16)

## **PHẦN 6 – ENCODER (LSTM)**

In [7]:
# ============================================================
# PHẦN 6 – ENCODER LSTM
# ============================================================

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, num_layers, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=PAD_IDX)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers,
                            dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lens):
        embedded = self.dropout(self.embedding(src))

        packed = pack_padded_sequence(embedded, src_lens.cpu(),
                                      batch_first=True, enforce_sorted=True)

        outputs, (hidden, cell) = self.lstm(packed)

        return hidden, cell


## **PHẦN 7 – DECODER (LSTM)**

In [8]:
# ============================================================
# PHẦN 7 – DECODER LSTM
# ============================================================

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, num_layers, dropout):
        super().__init__()

        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=PAD_IDX)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers,
                            dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_token, hidden, cell):
        # input_token shape: [batch]
        input_token = input_token.unsqueeze(1)  # -> [batch,1]
        embedded = self.dropout(self.embedding(input_token))

        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))

        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell


## **HẦN 8 – MODEL WRAPPER + TRAIN LOOP**

In [9]:
# ============================================================
# PHẦN 8 – SEQ2SEQ + TRAIN LOOP
# ============================================================

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_lens, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        output_dim = self.decoder.fc_out.out_features
        
        outputs = torch.zeros(batch_size, trg_len, output_dim).to(DEVICE)

        hidden, cell = self.encoder(src, src_lens)

        input_token = trg[:, 0]   # <sos>

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[:, t] = output

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)

            input_token = trg[:, t] if teacher_force else top1

        return outputs


# Khởi tạo model
INPUT_DIM = len(en_word2idx)
OUTPUT_DIM = len(de_word2idx)
EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
DROPOUT = 0.3

enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)

model = Seq2Seq(enc, dec).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

## **TRAINING FUNCTION**

In [10]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    epoch_loss = 0

    for src, trg, src_lens, _ in loader:
        src, trg = src.to(DEVICE), trg.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(src, src_lens, trg)

        # reshape for CE
        outputs = outputs[:, 1:].reshape(-1, OUTPUT_DIM)
        trg = trg[:, 1:].reshape(-1)

        loss = criterion(outputs, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(loader)


def evaluate(model, loader, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for src, trg, src_lens, _ in loader:
            src, trg = src.to(DEVICE), trg.to(DEVICE)

            outputs = model(src, src_lens, trg, teacher_forcing_ratio=0)

            outputs = outputs[:, 1:].reshape(-1, OUTPUT_DIM)
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(outputs, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(loader)


## **PHẦN 9 – TRAIN TOÀN BỘ MÔ HÌNH**

In [12]:
import time

N_EPOCHS = 10
best_val = float("inf")

train_losses = []
val_losses = []

print("===== TRAINING STARTED =====")

start_total = time.time()

for epoch in range(N_EPOCHS):

    epoch_start = time.time()

    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    val_loss = evaluate(model, val_loader, criterion)

    epoch_end = time.time()
    epoch_time = epoch_end - epoch_start
    elapsed = epoch_end - start_total
    remaining = (N_EPOCHS - (epoch + 1)) * epoch_time

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"\nEpoch {epoch+1}/{N_EPOCHS}")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Loss:   {val_loss:.4f}")
    print(f"  Time:       {epoch_time:.2f} sec")
    print(f"  Elapsed:    {elapsed/60:.2f} min")
    print(f"  ETA:        {remaining/60:.2f} min")

    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), "best_model_en_de.pth")
        print("  → Saved best model!")

print("\n===== TRAINING FINISHED =====")
print(f"Total time: {(time.time() - start_total)/60:.2f} min")


===== TRAINING STARTED =====


KeyboardInterrupt: 

## **PHẦN 10 – HÀM DỊCH translate(sentence)**

In [ ]:
def translate(sentence, max_len=50):
    model.eval()

    tokens = tokenize_en(sentence)
    ids = [en_word2idx.get(tok, UNK_IDX) for tok in tokens]
    src_tensor = torch.tensor(ids).unsqueeze(0).to(DEVICE)
    src_len = torch.tensor([len(ids)])

    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor, src_len)

    input_token = torch.tensor([de_word2idx[SOS_TOKEN]]).to(DEVICE)

    generated = []

    for _ in range(max_len):
        output, hidden, cell = model.decoder(input_token, hidden, cell)
        top1 = output.argmax(1).item()
        word = de_idx2word[top1]

        if word == EOS_TOKEN:
            break

        generated.append(word)
        input_token = torch.tensor([top1]).to(DEVICE)

    return " ".join(generated)


## **PHẦN 11 – TÍNH BLEU**

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def evaluate_bleu(model, loader):
    refs = []
    hyps = []

    for src, trg, src_lens, _ in loader:
        src = src.to(DEVICE)

        for i in range(src.size(0)):
            src_ids = src[i][:src_lens[i]].tolist()
            src_sentence = " ".join([en_idx2word[t] for t in src_ids])

            pred = translate(src_sentence)
            hyp = pred.split()

            trg_ids = trg[i].tolist()
            ref_sentence = [de_idx2word[t] for t in trg_ids
                            if t not in [PAD_IDX, SOS_IDX, EOS_IDX]]

            refs.append([ref_sentence])
            hyps.append(hyp)

    return corpus_bleu(refs, hyps) * 100

bleu_test = evaluate_bleu(model, test_loader)
print("BLEU on test:", bleu_test)


## **PHẦN 12 – BIỂU ĐỒ LOSS**

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train_losses, label="Train")
plt.plot(val_losses, label="Validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()
plt.show()

## **PHẦN 13 – PHÂN TÍCH LỖI**

In [ ]:
examples = [
    "a man with a red hat is riding a bicycle.",
    "two dogs are running in the field.",
    "a woman is sitting on the bench reading a book."
]

for s in examples:
    print("EN:", s)
    print("DE:", translate(s))
    print("-----")